In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import scipy.stats
from scipy.stats import ks_2samp, entropy
from scipy.spatial import distance
from sklearn.metrics import pairwise
from matplotlib.animation import FuncAnimation
import scipy
import seaborn as sns
from scipy.stats import anderson_ksamp



In [3]:
# Function to generate numeric distribution
def generate_distribution(size=100000, seed=1000):
    np.random.seed(seed)
    distribution1 = np.random.normal(loc=-3, scale=1, size=int(size * 0.2))
    distribution2 = np.random.normal(loc=2, scale=0.5, size=int(size * 0.2))
    distribution3 = np.random.normal(loc=5, scale=2, size=int(size * 0.2))
    distribution4 = np.random.exponential(scale=1, size=int(size * 0.2))
    distribution5 = np.random.uniform(low=-1, high=1, size=int(size * 0.2))
    return np.concatenate([distribution1, distribution2, distribution3, distribution4, distribution5])


def introduce_drift(original_distribution, drift_magnitude=0.0, frame=0, total_frames=5, undersamplefactor=1):
    # Simulate dynamic shift in means of normal distributions over frames
    mean_shift = drift_magnitude #* np.sin(2 * np.pi * frame / total_frames)
    distribution1 = np.random.normal(loc=-3 + mean_shift, scale=1, size=int(len(original_distribution) * 0.2)//undersamplefactor)
    distribution2 = np.random.normal(loc=2 + mean_shift, scale=0.5, size=int(len(original_distribution) * 0.2)//undersamplefactor)
    distribution3 = np.random.normal(loc=5 + mean_shift, scale=2, size=int(len(original_distribution) * 0.2)//undersamplefactor)
    distribution4 = np.random.exponential(scale=1, size=int(len(original_distribution) * 0.2)//undersamplefactor)
    distribution5 = np.random.uniform(low=-1, high=1, size=int(len(original_distribution) * 0.2)//undersamplefactor)

    return np.concatenate([distribution1, distribution2, distribution3, distribution4, distribution5])

In [4]:
# function to make pdf from samples
def probabilty_distribtion_from_samples(actual, expected, buckets=20 , epsilon=1e-11):
    
    # Cap data at 1st and 99th percentiles
    expected = np.clip(expected, np.percentile(expected, 1), np.percentile(expected, 99))
    actual = np.clip(actual, np.percentile(actual, 1), np.percentile(actual, 99))

    # Create bins for the variable
    bins = np.linspace(min(expected.min(), actual.min()), max(expected.max(), actual.max()), buckets + 1)

    # Discretize the variable into bins
    expected_bins = pd.cut(expected, bins, include_lowest=True)
    actual_bins = pd.cut(actual, bins, include_lowest=True)

    # Convert Categorical columns to Series
    expected_labels = expected_bins.astype(str)
    actual_labels = actual_bins.astype(str)

  
    # Find common labels using NumPy's intersect1d
    common_labels = np.intersect1d(np.unique(expected_labels), np.unique(actual_labels))

    if len(np.unique(expected_labels)) != len(np.unique(common_labels)):
        print("Warning: Expected labels do not match common labels")
        return probabilty_distribtion_from_samples(expected, actual, buckets=int(buckets/2), epsilon=1e-11)
    
    if len(np.unique(common_labels)) != len(np.unique(actual_labels)):
        print("Warning: Expected labels do not match common labels")
        actual_proportions,expected_proportions = probabilty_distribtion_from_samples(expected, actual, buckets=int(buckets/2), epsilon=1e-9)
        return probabilty_distribtion_from_samples(expected, actual, buckets=int(buckets/2), epsilon=1e-11)
    
    # Perform boolean indexing to get common bins
    expected_common = expected_bins[np.isin(expected_labels, common_labels)]
    actual_common = actual_bins[np.isin(actual_labels, common_labels)]

    # Calculate the proportion of observations in each bin for both datasets
    expected_proportions = (expected_common.value_counts().sort_index() + epsilon) / (len(expected_common) + epsilon*len(np.unique(common_labels)))
    actual_proportions = (actual_common.value_counts().sort_index() + epsilon) / (len(actual_common) + epsilon*len(np.unique(common_labels)))

 
    return actual_proportions,expected_proportions

In [5]:
# function to compute PSI
def calculate_psi(expected, actual, buckets=10, epsilon=1e-9):
    """
    Calculate the Population Stability Index (PSI) for a variable.

    Parameters:
        expected (array-like): Expected values (e.g., training dataset).
        actual (array-like): Actual values (e.g., test dataset).
        buckets (int): Number of bins for discretizing the variable.
        epsilon (float): Small constant to add to expected proportions.

    Returns:
        float: Population Stability Index (PSI).
    """

    actual_proportions, expected_proportions =  probabilty_distribtion_from_samples(actual, expected, buckets=buckets , epsilon=1e-9)
    
    # Calculate the PSI for each bin
    psi_values = (actual_proportions - expected_proportions) * np.log(actual_proportions / expected_proportions)

    # Sum the PSI values to get the overall PSI
    psi = psi_values.sum()

    return psi


In [6]:
# Function to compute metrics
def compute_metrics(reference_distribution_samples, variant_distribution_samples):
    # Two-sample mean z-test
    z_test, p_value = scipy.stats.ttest_ind(reference_distribution_samples, variant_distribution_samples)

    # Kolmogorov-Smirnov test
    ks_statistic, ks_p_value = ks_2samp(reference_distribution_samples, variant_distribution_samples)

    # Convert Samples to Probability Distributions
    ref_pdf, variant_pdf = probabilty_distribtion_from_samples(reference_distribution_samples, variant_distribution_samples)
    
    
    # KL Divergence
    kl_divergence = np.sum(variant_pdf * np.log(variant_pdf / ref_pdf))


    # JS Distance
    js_distance = distance.jensenshannon(ref_pdf, variant_pdf)

    # PSI Index
    psi_index = calculate_psi(reference_distribution_samples, variant_distribution_samples)


    # K-Sample Anderson-Darling
    result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


    return z_test, p_value, ks_statistic, ks_p_value, kl_divergence, js_distance, psi_index, result.significance_level




In [7]:
# Set parameters
sample_sizes = [10000, 25000, 50000, 100000, 250000, 500000, 1000000]
drift_magnitude_small = 0.05
drift_magnitude_drastic = 1.0


# CREATE empty dataframe to store all metrics
metric_values = pd.DataFrame(columns=['Drift Type','Sample Size', 'T Test (p-value)',  'KS (p-value)',  'AK (p-value)', 'KL Divergence', 'JS Distance', 'PSI'])

for i in range(0,10):
    print(i)
# Function to update the plots for each frame
    for sample_size in sample_sizes:
        #print(sample_size)
        reference_distribution = generate_distribution(size=sample_size)
        variant_distribution_small_drift = introduce_drift(reference_distribution, drift_magnitude_small)
        variant_distribution_drastic_drift = introduce_drift(reference_distribution, drift_magnitude_drastic)

        z_test_small_drift, p_value_small_drift, ks_statistic_small_drift, ks_p_value_small_drift, kl_divergence_small_drift, js_distance_small_drift, psi_index_small_drift , anderson_ksamp_small= compute_metrics(reference_distribution, variant_distribution_small_drift)
        z_test_drastic_drift, p_value_drastic_drift, ks_statistic_drastic_drift, ks_p_value_drastic_drift, kl_divergence_drastic_drift, js_distance_drastic_drift, psi_index_drastic_drift , anderson_ksamp_drastic= compute_metrics(reference_distribution, variant_distribution_drastic_drift)

        small = pd.DataFrame({'Drift Type':'Negligible Drift', 'Sample Size':sample_size,  
                              'T Test (p-value)':p_value_small_drift, 'KS (p-value)':ks_p_value_small_drift, 'AK (p-value)':anderson_ksamp_small,
                              'KL Divergence':kl_divergence_small_drift, 
                              'JS Distance':js_distance_small_drift, 'PSI':psi_index_small_drift}, index=[0])
        metric_values = pd.concat([metric_values,small], ignore_index=True)
        
        drastic = pd.DataFrame({'Drift Type':'Large Drift', 'Sample Size':sample_size, 
                                'T Test (p-value)':p_value_drastic_drift,  'KS (p-value)':ks_p_value_drastic_drift,  'AK (p-value)':anderson_ksamp_drastic,
                                'KL Divergence':kl_divergence_drastic_drift, 'JS Distance':js_distance_drastic_drift, 'PSI':psi_index_drastic_drift}, index=[0])
        metric_values = pd.concat([metric_values,drastic], ignore_index=True)

# group by drift type and sample size and compute all means and don't forget to reset the index
metric_values = metric_values.groupby(['Drift Type','Sample Size']).mean().reset_index()
metric_values

0
10000


C:\Users\salmank\AppData\Local\Temp\ipykernel_12904\905130667.py:25: UserWarning: p-value capped: true value larger than 0.25
  result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


C:\Users\salmank\AppData\Local\Temp\ipykernel_12904\905130667.py:25: UserWarning: p-value floored: true value smaller than 0.001
  result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


25000


C:\Users\salmank\AppData\Local\Temp\ipykernel_12904\905130667.py:25: UserWarning: p-value floored: true value smaller than 0.001
  result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


...

C:\Users\salmank\AppData\Local\Temp\ipykernel_12904\905130667.py:25: UserWarning: p-value floored: true value smaller than 0.001
  result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


1000000


C:\Users\salmank\AppData\Local\Temp\ipykernel_12904\905130667.py:25: UserWarning: p-value floored: true value smaller than 0.001
  result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


C:\Users\salmank\AppData\Local\Temp\ipykernel_12904\905130667.py:25: UserWarning: p-value floored: true value smaller than 0.001
  result = anderson_ksamp([reference_distribution_samples, variant_distribution_samples])


,Drift Type,Sample Size,T Test (p-value),KS (p-value),AK (p-value),KL Divergence,JS Distance,PSI
0,Large Drift,10000,1.198091e-47,1.106136e-101,0.001000,0.168074,0.193463,0.310323
1,Large Drift,25000,6.716136e-127,9.176624e-249,0.001000,0.178625,0.200113,0.333151
2,Large Drift,50000,1.301025e-234,0.000000e+00,0.001000,0.166433,0.193757,0.311386
3,Large Drift,100000,0.000000e+00,0.000000e+00,0.001000,0.169606,0.194859,0.315302
4,Large Drift,250000,0.000000e+00,0.000000e+00,0.001000,0.175437,0.198560,0.327580
5,Large Drift,500000,0.000000e+00,0.000000e+00,0.001000,0.173565,0.197542,0.324029
6,Large Drift,1000000,0.000000e+00,0.000000e+00,0.001000,0.175112,0.198365,0.326861
7,Negligible Drift,10000,9.063946e-01,8.127749e-01,0.250000,0.000955,0.015412,0.001131
8,Negligible Drift,25000,1.117020e-01,1.503627e-01,0.117579,0.000929,0.015256,0.001048
9,Negligible Drift,50000,1.633783e-01,7.480849e-02,0.040037,0.001155,0.016983,0.001554


In [8]:

# order by typ descending
metric_values = metric_values.sort_values(by=['Drift Type'], ascending=False)

# reindex dataframe
metric_values = metric_values.reset_index(drop=True)


# save to excel
metric_values.to_csv('metrics.csv')
